In [216]:
import requests 
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import time


#dictionary for data scraping
metacritic_dict = {'title':[], 'platform':[], 'release_date':[], 'metascore':[], 'critic_reviews':[],'pos_critic_count':[],'mix_critic_count':[],'neg_critic_count':[], 'user_score':[], 'user_ratings':[]}

# titles and platforms
titles = ["battlefield-1942","battlefield-vietnam","battlefield-2","battlefield-2142",
            "battlefield-bad-company","battlefield-bad-company-2","battlefield-3","battlefield-4",
            "battlefield-hardline","battlefield-v","battlefield-1"]
platforms = ["pc", "playstation-3","playstation-4","xbox-360", "xbox-one"]
user_agent = {'User-agent': 'Mozilla/5.0'}

for i in range(0,len(titles)):
    for j in range(0,len(platforms)):
             
        url = 'https://www.metacritic.com/game/'+platforms[j]+'/'+titles[i]
        response  = requests.get(url, headers = user_agent)

        
        if response.status_code == 200: #if the combination of title and platform exists

            #title and platform
            metacritic_dict['title'].append(titles[i])
            metacritic_dict['platform'].append(platforms[j])

            soup = BeautifulSoup(response.text, 'html.parser')
            url_base = '/game/'+platforms[j]+'/'+titles[i]
            
            #metascore
            critic_score = soup.find('a', class_='metascore_anchor', href=url_base+'/critic-reviews')
            if critic_score == None:
                metacritic_dict['metascore'].append(int(0))
            else:
                metacritic_dict['metascore'].append(int(critic_score.find('span').text))
        
            #user score
            user_score = soup.find('a', class_='metascore_anchor', href=url_base+'/user-reviews')
            if user_score == None:
                metacritic_dict['user_score'].append(float(0))
            else:
                metacritic_dict['user_score'].append(float(user_score.find('div').text))
            
            #release_date
            release_date = soup.find('li', class_='summary_detail release_data')
            if release_date == None:
                metacritic_dict['release_date'].append(float(0))
            else:
                release_date = datetime.strptime(release_date.find('span', class_='data').text,'%b %d, %Y')
                metacritic_dict['release_date'].append(release_date)

            url_base_count = url_base +'/critic-reviews?dist='
            
            #number of positive critic reviews
            pos_critic_count = soup.find('a', href=url_base_count+'positive')
            if pos_critic_count == None:
                metacritic_dict['pos_critic_count'].append(int(0))
            else:
                metacritic_dict['pos_critic_count'].append(int(pos_critic_count.find('span', class_="count").text))
            
            #number of neutral critic reviews
            mix_critic_count = soup.find('a', href=url_base_count+'neutral')
            if mix_critic_count == None:
                metacritic_dict['mix_critic_count'].append(int(0))
            else:
                metacritic_dict['mix_critic_count'].append(int(mix_critic_count.find('span', class_="count").text))
            
            #number of negative critic reviews
            neg_critic_count = soup.find('a', href=url_base_count+'negative')
            if neg_critic_count == None:
                metacritic_dict['neg_critic_count'].append(int(0))
            else:
                metacritic_dict['neg_critic_count'].append(int(neg_critic_count.find('span', class_="count").text))
            
            #number of user reviews
            for summary in soup.find_all('div', class_='summary'):
                if summary.find('a') == None:
                    break
                if summary.find('a', href=url_base+'/critic-reviews'):
                    critic_reviews = summary.find('a', href=url_base+'/critic-reviews').find('span').text
                    metacritic_dict['critic_reviews'].append(int(critic_reviews))
                elif summary.find('a', href=url_base+'/user-reviews'):
                    user_reviews = summary.find('a', href=url_base+'/user-reviews')
                    metacritic_dict['user_ratings'].append(int(user_reviews.text.replace(' Ratings','')))       

bf_metacritic = pd.DataFrame(metacritic_dict, columns = ['title', 'platform', 'release_date', 'metascore', 'critic_reviews','pos_critic_count','mix_critic_count','neg_critic_count', 'user_score', 'user_ratings'])

bf_metacritic

,title,platform,release_date,metascore,critic_reviews,pos_critic_count,mix_critic_count,neg_critic_count,user_score,user_ratings
0,battlefield-1942,pc,2002-09-10,89,27,27,0,0,8.6,428
1,battlefield-vietnam,pc,2004-03-14,84,48,45,3,0,7.6,163
2,battlefield-2,pc,2005-06-21,91,55,54,1,0,8.4,1005
3,battlefield-2142,pc,2006-10-17,80,48,41,7,0,6.8,354
4,battlefield-bad-company,playstation-3,2008-06-23,84,50,48,2,0,8.1,244
5,battlefield-bad-company,xbox-360,2008-06-23,83,70,61,9,0,7.7,286
6,battlefield-bad-company-2,pc,2010-03-02,87,46,45,1,0,8.3,1721
7,battlefield-bad-company-2,playstation-3,2010-03-02,88,59,58,1,0,8.5,622
8,battlefield-bad-company-2,xbox-360,2010-03-02,88,75,72,3,0,8.6,925
9,battlefield-3,pc,2011-10-25,89,61,60,1,0,7.6,5277


In [217]:
Date_Time = time.strftime("%d%m%Y-%H%M%S")
excelfilename = 'bf_metacritic_'+Date_Time +".xlsx"
bf_metacritic.to_excel (r'/Users/liuba/Desktop/files/'+excelfilename, index = False, header=True)